In [2]:
%pwd
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [3]:
# stdlib
import os
import warnings
from datetime import datetime
from itertools import product

# third party
import click
import neptune
import numpy as np
import pandas as pd

# first party
from config import (
    low_dimensional_problem_names,
    search_agg_functions,
    search_hidden_activations,
    search_hidden_sizes,
    search_include_bn,
    search_input_sizes,
    search_output_activations,
)
from deeplifting.optimization import (
    run_deeplifting,
)
from deeplifting.problems import PROBLEMS_BY_NAME

# Filter warnings
warnings.filterwarnings('ignore')

In [ ]:
PROBLEMS = PROBLEMS_BY_NAME
problem_names = ['ackley']

# Get the available configurations
combinations = (
    search_input_sizes,
    search_hidden_sizes,
    search_hidden_activations,
    search_output_activations,
    search_agg_functions,
    search_include_bn,
)
configurations = list(product(*combinations))
trials = 10

# List to store performance data
performance_df_list = []
experiment_date = datetime.today().strftime('%Y-%m-%d-%H')
for problem_name in problem_names:
    # Run over the experiments
    for (
        index,
        (
            input_size,
            hidden_size,
            hidden_activation,
            output_activation,
            agg_function,
            include_bn,
        ),
    ) in enumerate(configurations):
        print(problem_name)

        # Load the problems
        problem = PROBLEMS[problem_name]
        print(
            input_size,
            hidden_size,
            hidden_activation,
            output_activation,
            agg_function,
            include_bn,
        )

        # Get the outputs
        outputs = run_deeplifting(
            problem,
            problem_name=problem_name,
            trials=trials,
            input_size=input_size,
            hidden_sizes=hidden_size,
            activation=hidden_activation,
            output_activation=output_activation,
            agg_function=agg_function,
            include_bn=include_bn,
            method='single-value',
        )

        # Get the results of the outputs
        output_size = problem['dimensions']
        x_columns = [f'x{i + 1}' for i in range(output_size)]
        columns = x_columns + ['f', 'f_initial', 'algorithm', 'total_time']

        results = pd.DataFrame(outputs['final_results'], columns=columns)

        # Add meta data to the results
        results['input_size'] = input_size
        results['hidden_size'] = '-'.join(map(str, hidden_size))
        results['num_layers'] = len(hidden_size)
        results['num_neurons'] = hidden_size[0]
        results['hidden_activation'] = hidden_activation
        results['output_activation'] = output_activation
        results['agg_function'] = agg_function
        results['include_bn'] = include_bn
        results['problem_name'] = problem_name
        results['global_minimum'] = problem['global_minimum']
        results['dimensions'] = output_size
        results['hits'] = np.abs(results['f'] - results['global_minimum']) <= 1e-4

        # Print the results
        hits = results['hits'].mean()
        run_time = results['total_time'].mean()
        print(f'Success Rate = {hits}')
        print(f'Average run time = {run_time}')

        # Save to parquet
        layers = len(hidden_size)
        units = hidden_size[0]

        path = f'./{directory}/{experiment_date}-{problem_name}'
        if not os.path.exists(path):
            os.makedirs(path)

        # Append performance
        performance_df_list.append(results)

        if hits >= 0.90:
            break